In [33]:
from tqdm.auto import tqdm
from call_model import call_chat_once
import asyncio
import httpx
from uuid import uuid4
import json
import datasets
from typing import Optional
import os
import csv
import nest_asyncio
nest_asyncio.apply()

In [34]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train") # or load from data/test.csv
print(eval_dataset)

Using the latest cached version of the dataset since m-ric/huggingface_doc_qa_eval couldn't be found on the Hugging Face Hub
2025-08-15 11:35:05,550 - WARNING - Using the latest cached version of the dataset since m-ric/huggingface_doc_qa_eval couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/minhthuy/.cache/huggingface/datasets/m-ric___huggingface_doc_qa_eval/default/0.0.0/5f70aa9a1e2430f528ac3f27f01f0ba8719c0704 (last modified on Mon Aug 11 14:41:56 2025).
2025-08-15 11:35:05,552 - WARNING - Found the latest cached dataset configuration 'default' at /home/minhthuy/.cache/huggingface/datasets/m-ric___huggingface_doc_qa_eval/default/0.0.0/5f70aa9a1e2430f528ac3f27f01f0ba8719c0704 (last modified on Mon Aug 11 14:41:56 2025).


Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'standalone_score', 'standalone_eval', 'relatedness_score', 'relatedness_eval', 'relevance_score', 'relevance_eval'],
    num_rows: 65
})


In [35]:
OUTPUT_FILE = "data/rag_results_default_LLMListwiseRerank_gpt4omini__k_retriever=15.csv"

def load_existing_results(file_path):
    dataset = []
    existing_questions = set()
    if os.path.exists(file_path):
        with open(file_path, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_questions.add(row["user_input"])
                # parse retrieved_contexts lại từ JSON
                row["retrieved_contexts"] = json.loads(row["retrieved_contexts"])
                dataset.append(row)
    return existing_questions, dataset


def append_result_to_csv(file_path, row, fieldnames):
    file_exists = os.path.exists(file_path)
    with open(file_path, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    verbose: Optional[bool] = False
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    fieldnames = ["user_input", "retrieved_contexts", "response", "reference"]

    processed_questions, dataset = load_existing_results(OUTPUT_FILE)
    print(f"Đã có {len(processed_questions)} câu hỏi xử lý trước đó, sẽ bỏ qua chúng.")


    for i, example in enumerate(tqdm(eval_dataset)):
        question = example["question"]

        if question in processed_questions:
            continue
        
        payload = {
            "question": question,
            "session_id": str(uuid4()),
            "chat_history": []
        }

        try:
            relevant_docs, answer = asyncio.run(call_chat_once(payload))
        except Exception as e:
            print(f"Error from question: '{question}': {e}")

        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')

        row = {
            "user_input": question,
            "retrieved_contexts": json.dumps(
                [doc['kwargs'].get('page_content') for doc in relevant_docs],
                ensure_ascii=False
            ),
            "response": answer,
            "reference": example['answer']
        }
        append_result_to_csv(OUTPUT_FILE, row, fieldnames)
        dataset.append({
            "user_input": question,
            "retrieved_contexts": json.loads(row["retrieved_contexts"]),
            "response": answer,
            "reference": example['answer']
        })
    return dataset

In [36]:
data = run_rag_tests(eval_dataset=eval_dataset)


Đã có 0 câu hỏi xử lý trước đó, sẽ bỏ qua chúng.


  2%|▏         | 1/65 [00:03<04:15,  4.00s/it]

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What architecture is the `tokenizers-linux-x64-musl` binary designed for?\n', 'type': 'human', 'id': 'f535cfc3-3bdf-40fe-a3c6-092ff00f3552'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_bDlhmNtezppBkgNOw0zfmewh', 'function': {'arguments': '{"query":"tokenizers-linux-x64-musl architecture"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--2399b6b2-8c0b-4848-a231-471aa75b1d58-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'tokenizers-linux-x64-musl architecture'}, 'id': 'call_bDlhmNtezppBkgNOw0zfmewh', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 365, 'o

2025-08-15 11:35:14,788 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
  3%|▎         | 2/65 [00:09<04:56,  4.71s/it]

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the purpose of the BLIP-Diffusion model?\n', 'type': 'human', 'id': '00a88708-a0d3-47fb-ac68-0a511277bb18'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_oYS7Uoe3W31nKJupiyz8vX55', 'function': {'arguments': '{"query":"BLIP-Diffusion model purpose"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--92541e2a-f2b5-4712-9e43-2b8272db375a-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'BLIP-Diffusion model purpose'}, 'id': 'call_oYS7Uoe3W31nKJupiyz8vX55', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 361, 'output_tokens': 22, 'total_tokens': 383, 'inpu

...

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the name of the open-source library created by Hugging Face to simplify Transformer acceleration?\n', 'type': 'human', 'id': '88e15205-c546-471b-8ca3-f96ef507f44a'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_KRYFZirJ41oEdpivl7nHvpcg', 'function': {'arguments': '{"query":"open-source library created by Hugging Face to simplify Transformer acceleration"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--7ff7a1c3-66f8-494b-967d-6838bd498eb3-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'open-source library created by Hugging Face to simplify Transformer acce

2025-08-15 11:40:13,112 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
 98%|█████████▊| 64/65 [05:07<00:05,  5.08s/it]

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What parameter is used to ensure that elements in a row have the same height in Gradio?\n', 'type': 'human', 'id': 'fd99de7d-1787-491f-808d-9985d1ccc4f4'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_kylEiYO25PLE5ZRpY6ZkPoww', 'function': {'arguments': '{"query":"Gradio same height elements in a row parameter"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--805e1346-1047-4f27-8ea0-d83a9e785940-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'Gradio same height elements in a row parameter'}, 'id': 'call_kylEiYO25PLE5ZRpY6ZkPoww', 'type': 'tool_call'}], 'usage_metad

2025-08-15 11:40:17,155 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
100%|██████████| 65/65 [05:11<00:00,  4.79s/it]

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the command to install the latest version of Optimum with OpenVINO support?\n', 'type': 'human', 'id': 'ac9f7a7e-c43f-4658-a725-0b98e3bcd136'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_qxvSKmiv4hoIv2WzraLeg8k7', 'function': {'arguments': '{"query":"install latest version of Optimum with OpenVINO support"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--cbe41801-bfa3-446a-90b3-66ed0dcc3b06-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'install latest version of Optimum with OpenVINO support'}, 'id': 'call_qxvSKmiv4hoIv2WzraLeg8k7', 'type': 'tool_call'}]

In [37]:
from ragas import EvaluationDataset
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    FactualCorrectness
)
evaluation_dataset = EvaluationDataset.from_list(data)

# Khởi tạo các metric
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]
# metrics = [FactualCorrectness()]

In [38]:
print(len(evaluation_dataset))

65


In [39]:
from app.config import configs

import os
os.environ['OPENAI_API_KEY'] = configs.OPENAI_API_KEY

result = evaluate(
    evaluation_dataset,
    metrics=metrics,
)

print(result) 

Evaluating:   0%|          | 0/260 [00:00<?, ?it/s]2025-08-15 11:40:20,022 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-15 11:40:20,026 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-15 11:40:20,033 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-15 11:40:20,037 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-15 11:40:20,043 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-15 11:40:20,051 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-15 11:40:20,054 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-15 11:40:20,657 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-15 11:40:21,352 - INFO - HTTP Request: POST

{'faithfulness': 0.9009, 'answer_relevancy': 0.9495, 'context_precision': 0.8962, 'context_recall': 0.8987}
